# GPU speed comparison

In [1]:
import sys
import os


root_dir_name = "ML_competitions"
cwd = os.getcwd()

if root_dir_name in cwd:
    project_root = cwd[:cwd.index(root_dir_name) + len(root_dir_name)]
    
    if project_root not in sys.path:
        sys.path.append(project_root)
        print(f"Added to sys.path: {project_root}")
else:
    raise RuntimeError(f"'{root_dir_name} not found in current path")

Added to sys.path: /Users/kentookubo/Documents/Projects/ML_competitions


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

from my_library.models.custom_tabn import CustomTabNetModel
from my_library.configs.model_configs.tabn_configs import TabNetConfig
from my_library.configs.model_configs.fit_configs import FitConfig
from my_library.validations.validation_runner import ValidationRunner
from my_library.parameter_tunings.optuna_tuner import OptunaValidationTuner
from my_library.utils.timeit import timeit
from my_library.splitters.cross_validation import CrossValidationSplitter

from sklearn.metrics import roc_auc_score

In [3]:
# 擬似巨大データの生成
X, y = make_classification(n_samples=100_000, n_features=200, n_informative=50, n_redundant=50, random_state=42)
X = pd.DataFrame(X, columns=[f"feat_{i}" for i in range(X.shape[1])])
y = pd.Series(y, name="target")

In [4]:
X.shape, y.shape

((100000, 200), (100000,))

In [5]:
# 共通設定
feats = X.columns.tolist()
fit_config = FitConfig(feats=feats, early_stopping_rounds=10, epochs=100, batch_size=1024)
folds = CrossValidationSplitter(n_splits=5, shuffle=True, random_state=42).split(X, y)

In [6]:
# 実行関数の定義
@timeit
def run_runner(use_gpu: bool, parallel_mode: str):
    config = TabNetConfig(
        task_type="classification",
        params=dict(n_d=8, n_a=8, n_steps=3, batch_size=1024, max_epochs=100),
        use_gpu=use_gpu
    )
    runner = ValidationRunner(
        model_class=CustomTabNetModel,
        model_configs=config,
        metric_fn=roc_auc_score,
        predict_proba=True,
        parallel_mode=parallel_mode
    )
    results = runner.run(folds, fit_config)
    return results

In [7]:
# 時間計測（3パターン）
results_cpu_serial = run_runner(use_gpu=False, parallel_mode=False)
results_cpu_parallel = run_runner(use_gpu=False, parallel_mode=True)
# results_gpu_serial = run_runner(use_gpu=True, parallel_mode=False)

2025-05-06 18:24:49,479 - my_library.utils.timeit - INFO - [Start] run_runner
2025-05-06 18:24:49,480 - my_library.validations.validation_runner - INFO - Initialized ValidationRunner | model=CustomTabNetModel task=classification
2025-05-06 18:24:49,481 - my_library.utils.timeit - INFO - [Start] ValidationRunner.run
2025-05-06 18:24:49,481 - my_library.validations.validation_runner - INFO - Validation run started.
2025-05-06 18:24:49,482 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cpu.
2025-05-06 18:24:49,483 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cpu.
2025-05-06 18:24:49,483 - my_library.validations.validation_runner - INFO - [Fold 1] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=False
2025-05-06 18:24:49,484 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-06 18:24:49,485 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samp


Early stopping occurred at epoch 34 with best_epoch = 14 and best_valid_accuracy = 0.99165


/Users/kentookubo/Documents/Projects/ML_competitions/kaggle-venv/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-06 18:28:48,850 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-06 18:28:48,875 - my_library.validations.validator - INFO - Training completed.
2025-05-06 18:28:49,745 - my_library.validations.validation_runner - INFO - [Fold 1] Score=0.9916
2025-05-06 18:28:49,751 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cpu.
2025-05-06 18:28:49,752 - my_library.validations.validation_runner - INFO - [Fold 2] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=False
2025-05-06 18:28:49,752 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-06 18:28:49,753 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200



Early stopping occurred at epoch 37 with best_epoch = 17 and best_valid_accuracy = 0.9924


/Users/kentookubo/Documents/Projects/ML_competitions/kaggle-venv/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-06 18:33:10,562 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-06 18:33:10,581 - my_library.validations.validator - INFO - Training completed.
2025-05-06 18:33:11,521 - my_library.validations.validation_runner - INFO - [Fold 2] Score=0.9924
2025-05-06 18:33:11,525 - CustomTabNetModel - INFO - Initialized CustomTabNetModel on device=cpu.
2025-05-06 18:33:11,525 - my_library.validations.validation_runner - INFO - [Fold 3] Train=(80000, 200), Valid=(20000, 200),                 use_gpu=False
2025-05-06 18:33:11,526 - my_library.validations.validator - INFO - Initialized Validator for model: CustomTabNetModel
2025-05-06 18:33:11,527 - my_library.validations.validator - INFO - Training start | model=CustomTabNetModel samples=80000 features=200


...

/Users/kentookubo/Documents/Projects/ML_competitions/kaggle-venv/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-06 19:22:27,937 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-06 19:22:27,990 - my_library.validations.validator - INFO - Training completed.
2025-05-06 19:22:31,071 - my_library.validations.validation_runner - INFO - [Fold 5] Score=0.9924



Early stopping occurred at epoch 56 with best_epoch = 36 and best_valid_accuracy = 0.9923


/Users/kentookubo/Documents/Projects/ML_competitions/kaggle-venv/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 56 with best_epoch = 36 and best_valid_accuracy = 0.9913


/Users/kentookubo/Documents/Projects/ML_competitions/kaggle-venv/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2025-05-06 19:25:33,978 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-06 19:25:34,020 - my_library.validations.validator - INFO - Training completed.
2025-05-06 19:25:34,308 - CustomTabNetModel - INFO - TabNet training completed.
2025-05-06 19:25:34,344 - my_library.validations.validator - INFO - Training completed.
2025-05-06 19:25:36,260 - my_library.validations.validation_runner - INFO - [Fold 3] Score=0.9923
2025-05-06 19:25:36,359 - my_library.validations.validation_runner - INFO - [Fold 4] Score=0.9913
2025-05-06 19:25:36,371 - my_library.validations.validation_runner - INFO - Validation completed | mean=0.9922 std=0.0005
2025-05-06 19:25:36,372 - my_library.utils.timeit - INFO - [End] ValidationRunner.run - Elapsed: 2142.0503 sec
2025-05-06 19:25:

In [8]:
# OptunaによるGPUチューニング
@timeit
def run_gpu_optuna():
    config = TabNetConfig(
        task_type="classification",
        params={},  # 初期は空でもOK
        use_gpu=True
    )
    tuner = OptunaValidationTuner(
        model_class=CustomTabNetModel,
        model_configs=config,
        param_space={
            "n_d": [8, 16],
            "n_steps": [3, 5],
            "learning_rate": (0.01, 0.2),
        },
        folds=folds,
        scoring=roc_auc_score,
        predict_proba=True,
        n_trials=5,
        maximize=True,
        parallel_mode=False,
    )
    return tuner.tune(fit_config, return_runner_results=True)


In [9]:
# best_params, best_score, best_result = run_gpu_optuna()